In [ ]:
NGROK_TOKEN = "2fUd1uDXbhiJpAEmajtGUeGAZNN_3y9fpYRLaqvNiuBLG9juE"

!pip install flask
!pip install pyngrok

from pyngrok import ngrok, conf
ngrok.set_auth_token(NGROK_TOKEN)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp '/content/drive/MyDrive/Module3' -r '/content/'

In [ ]:
%cd '/content/Module3/'

/content/Module3


In [ ]:
!pip install gtts SpeechRecognition pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 28.7 MB/s eta 0:00:00


In [ ]:
from flask import Flask, render_template, jsonify,redirect, url_for, request
import json
import os
from gtts import gTTS
import speech_recognition as sr
from transformers import pipeline
import json
import random
import utils
from pydub import AudioSegment
from pyngrok import ngrok, conf

conf.get_default().auth_token = NGROK_TOKEN


app = Flask(__name__)

sentiment_model = pipeline('sentiment-analysis')


with open('details.json', 'r') as f:
    users = json.load(f)


@app.route('/', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        global latitude
        global longitude

        # data = request.get_json()
        longitude = request.form['longitude']
        latitude = request.form['latitude']

        username = request.form['email']
        password = request.form['password']

        print('user {username} password {password} ')

        if username in users['login'].keys() and users['login'][username] == password:
            return redirect(url_for('chat'))

        error = 'Invalid username or password'
        return render_template('Login.html', error=error)

    return render_template('Login.html')


@app.route('/signup', methods=['GET', 'POST'])
def signup():
    if request.method == 'POST':
        email = request.form['email']
        password = request.form['password']
        confirmPassword = request.form['confirm-password']

        print('user {email} password {password} confirm passs {confirmPassword}')

        users['login'][email] = password
        with open('details.json', 'w') as jf:
            json.dump(users, jf)

        return redirect(url_for('login'))

    return render_template('Signup.html')


@app.route('/get_questions', methods=['GET'])
def get_questions():
    questions = [
        'Hi, How can I help you?',
        'Express your feelings to me, I\'m here to listen!',
        'My model is analyzing your texts, you want anything to add? If no please enter NA'
    ]
    return jsonify(questions)


@app.route('/chat', methods=['POST', 'GET'])
def chat():
    states = {0:'From your texts, our model estimates that you in a traumatic state. Please visit the doctor connect page and book a consultation',
              1:"From your texts, our model estimates that you are in a healthy state"}

    if request.method == 'POST':
        data = request.get_json()
        answers = data.get('answers')
        trauma_state_chat = utils.detect_trauma(" ".join(answers))
        response = {'msg': states.get(int(trauma_state_chat))}
        return jsonify(response)

    return render_template('Chat.html')


@app.route('/voice', methods=['POST', 'GET'])
def voice():

    states = {0:'From your audio feed, our model estimates that you in a traumatic state. Please visit the doctor connect page and book a consultation',
              1:"From your audio feed, our model estimates that you are in a healthy state"}

    if request.method == 'POST':
        # Check if the 'audioFile' field is in the request form data
        if 'audioFile' not in request.files:
            return redirect(request.url)

        audio_obj = request.files['audioFile']

        if audio_obj and audio_obj.filename:
            filename = os.path.join("/content/Module3", audio_obj.filename)
            audio_obj.save(filename)

            sound = AudioSegment.from_mp3(filename)
            sound.export("transcript.wav", format="wav")

            audio_text = utils.audio_to_text("transcript.wav")

        trauma_state = utils.detect_trauma(audio_text)
        response = {'msg': states.get(int(trauma_state))}
        print(response)
        return jsonify(response)  # Return the response as JSON

    return render_template('Voice.html')


@app.route('/doctor')
def doctor():
    res = utils.get_nearest(latitude, longitude)
    address = res['results'][0]['address']
    distance = "{0:.2f} Km".format(res['results'][0]['distance']/100)
    m = utils.create_map(res)
    dc_name, dc_email = utils.fetch_details()
    response = {'map':m._repr_html_(), 'name': dc_name, 'address':address, 'distance':distance}
    if response:
        send_email(dc_name, dc_email)

    return render_template('Doctor.html', map=response['map'], name = response['name'], address=address, distance=distance)


@app.route('/email')
def send_email(dc_name, dc_email):
    utils.send_email(dc_email)
    return


@app.route('/patient' , methods=['POST', 'GET'])
def patient():
    if request.method == 'POST':
        data = request.get_json()  # Get the JSON data from the request
        print("Data received from the form:")
        print(data)

        with open('patient_data.json', 'w') as jfk:
            json.dump(data, jfk)

        response = {'message': 'Data received and saved  successfully'}
        return jsonify(response)


    #frontend will send the patient details
    return render_template('Patient.html')


ngrok_tunnel = ngrok.connect(5000)
print(ngrok_tunnel)
# print('Running at:', ngrok_tunnel.public_url)

if __name__ == '__main__':
    app.run(port=5000)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:47:01] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:47:02] "GET /static/css/Login.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:47:03] "GET /static/assets/login.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:47:08] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:47:19] "POST /map_2 HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:47:26] "POST / HTTP/1.1" 302 -


user {username} password {password} 


INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:47:27] "GET /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:47:27] "GET /static/assets/user.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:47:27] "GET /static/css/Chat.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:47:28] "GET /static/assets/chat-background.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:47:28] "GET /get_questions HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:47:31] "GET /patient HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:47:32] "GET /static/css/Patient.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:47:45] "GET /doctor HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:47:46] "GET /static/css/Doctor.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:48:56] "GET /voice HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:48:57] "GET /static/css/Voice.css HTTP/1.1" 200 -
INFO:werkzeug:

{'msg': 'From your audio feed, our model estimates that you are in a healthy state'}


INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:49:53] "GET /voice HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:49:54] "GET /static/css/Voice.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:49:54] "GET /static/assets/voice-bg.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:51:43] "GET /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:51:43] "GET /static/assets/user.jpg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:51:43] "GET /static/css/Chat.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:51:44] "GET /get_questions HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:51:44] "GET /static/assets/chat-background.jpg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:51:54] "GET /voice HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:51:55] "GET /static/css/Voice.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2024 15:51:55] "GET /static/assets/voice-bg.png HTTP/1.1"